# Analyzing Telemetry Data

This notebook shows how to work with the telemetry data generated by simulations.

## What is Telemetry?

Telemetry data simulates sensor readings from a smartphone in a moving car:
- GPS coordinates (latitude, longitude)
- Accelerometer data (x, y, z axes)
- Gyroscope data (rotational motion)
- Timestamps

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from cascabel.models.waitline import WaitLine
from cascabel.models.simulation import Simulation
from cascabel.models.models import BorderCrossingConfig, SimulationConfig, PhoneConfig
from cascabel.simulation.csv_generator import CSVGenerator

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

## Generate Telemetry Data

In [ ]:
# Run simulation with telemetry
waitline = WaitLine(
    geojson_path="cascabel/paths/usa2mx/bota.geojson",
    speed_regime={"slow": 0.8, "fast": 0.2},
    line_length_seed=0.5
)

border_config = BorderCrossingConfig(
    num_queues=1,
    nodes_per_queue=[1],
    arrival_rate=0.5,
    service_rates=[0.8],
    queue_assignment='shortest',
    safe_distance=8.0,
    max_queue_length=50
)

simulation_config = SimulationConfig(
    max_simulation_time=1800.0,
    time_factor=1.0,
    enable_telemetry=True,
    enable_position_tracking=True
)

phone_config = PhoneConfig(
    sampling_rate=10.0,  # 10 Hz
    accelerometer_noise=0.01,
    gyroscope_noise=0.001,
    gps_accuracy=3.0
)

simulation = Simulation(
    waitline=waitline,
    border_config=border_config,
    simulation_config=simulation_config,
    phone_config=phone_config
)

simulation()
print("Simulation completed with telemetry generation")

## Load Telemetry Data

In a real scenario, you'd load from a CSV file. Here we'll extract from the simulation.

In [ ]:
# Get telemetry data from simulation
# Note: This depends on how telemetry is stored in the simulation object
# For demo, let's assume we have a way to get it

# For now, let's look at position tracking data
stats = simulation.get_statistics()

print(f"Total positions recorded: {stats.total_positions_recorded}")

# If telemetry CSV was generated, we could load it like:
# df = pd.read_csv('telemetry_output.csv')
# Let's simulate some telemetry data

np.random.seed(42)
n_samples = 1000
time = np.linspace(0, 180, n_samples)

# Simulate GPS path along the waitline
path_points = waitline.path
path_lats = [p[1] for p in path_points]
path_lons = [p[0] for p in path_points]

# Interpolate along path
t_interp = np.linspace(0, 1, n_samples)
lats = np.interp(t_interp, np.linspace(0, 1, len(path_lats)), path_lats)
lons = np.interp(t_interp, np.linspace(0, 1, len(path_lons)), path_lons)

# Add GPS noise
lats += np.random.normal(0, 0.0001, n_samples)  # ~10m noise
lons += np.random.normal(0, 0.0001, n_samples)

# Simulate accelerometer (smooth acceleration/deceleration)
speed = 10 + 5 * np.sin(2 * np.pi * time / 60)  # varying speed
accel_x = np.gradient(speed) + np.random.normal(0, 0.1, n_samples)
accel_y = np.random.normal(0, 0.05, n_samples)
accel_z = 9.8 + np.random.normal(0, 0.1, n_samples)  # gravity + noise

# Create DataFrame
telemetry_df = pd.DataFrame({
    'timestamp': time,
    'latitude': lats,
    'longitude': lons,
    'accel_x': accel_x,
    'accel_y': accel_y,
    'accel_z': accel_z,
    'speed': speed
})

print("Sample telemetry data:")
telemetry_df.head()

## GPS Track Visualization

In [ ]:
# Plot GPS track
fig, ax = plt.subplots()

scatter = ax.scatter(telemetry_df['longitude'], telemetry_df['latitude'], 
                    c=telemetry_df['timestamp'], cmap='viridis', s=10, alpha=0.7)
ax.plot(path_lons, path_lats, 'r--', linewidth=2, label='True Path')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('GPS Track with Color-coded Time')
ax.legend()
plt.colorbar(scatter, label='Time (s)')
plt.show()

## Accelerometer Data Analysis

In [ ]:
# Plot accelerometer data
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

axes[0].plot(telemetry_df['timestamp'], telemetry_df['accel_x'], 'b-', label='X-axis')
axes[0].set_ylabel('Acceleration (m/s²)')
axes[0].set_title('Accelerometer X-axis')
axes[0].grid(True)

axes[1].plot(telemetry_df['timestamp'], telemetry_df['accel_y'], 'g-', label='Y-axis')
axes[1].set_ylabel('Acceleration (m/s²)')
axes[1].set_title('Accelerometer Y-axis')
axes[1].grid(True)

axes[2].plot(telemetry_df['timestamp'], telemetry_df['accel_z'], 'r-', label='Z-axis')
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('Acceleration (m/s²)')
axes[2].set_title('Accelerometer Z-axis (includes gravity)')
axes[2].grid(True)

plt.tight_layout()
plt.show()

## Speed vs Acceleration Correlation

In [ ]:
# Plot speed vs acceleration
fig, ax = plt.subplots()

scatter = ax.scatter(telemetry_df['speed'], telemetry_df['accel_x'], 
                    c=telemetry_df['timestamp'], cmap='plasma', alpha=0.6)
ax.set_xlabel('Speed (m/s)')
ax.set_ylabel('Acceleration X (m/s²)')
ax.set_title('Speed vs Acceleration Correlation')
ax.grid(True)
plt.colorbar(scatter, label='Time (s)')

plt.show()

# Calculate correlation
correlation = telemetry_df['speed'].corr(telemetry_df['accel_x'])
print(f"Correlation between speed and acceleration: {correlation:.3f}")

## Real Telemetry Data Comparison

Let's compare with real telemetry data from the raw_data folder.

In [ ]:
# Load real data
real_df = pd.read_csv('raw_data/my_iOS_device_2017-12-12_14-36-55_-0700.csv')
print("Real telemetry data columns:")
print(real_df.columns.tolist())
print("\nSample real data:")
real_df.head()

In [ ]:
# Plot real vs simulated GPS
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Real data
ax1.scatter(real_df['locationLongitude(WGS84)'], real_df['locationLatitude(WGS84)'], 
           s=5, alpha=0.7, label='Real GPS')
ax1.set_title('Real GPS Track')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.grid(True)

# Simulated data
ax2.scatter(telemetry_df['longitude'], telemetry_df['latitude'], 
           s=5, alpha=0.7, c='orange', label='Simulated GPS')
ax2.set_title('Simulated GPS Track')
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')
ax2.grid(True)

plt.tight_layout()
plt.show()